In [14]:
import numpy as np
import pandas as pd

graph = np.array(
    [
        [0,8,15,10,21,26],
        [8,0,12,7,16,23],
        [15,12,0,19,9,11],
        [10,7,19,0,11,17],
        [21,16,9,11,0,13],
        [26,23,11,17,13,0]
    ]
)
demand = np.array([10,8,22,18,7,55])
distance = 10
P = 2
inf=1e9
upper_bound=[]
lower_bound=[]

a_ij = np.where(graph<=10,1,0)
X_list = np.array([1,1,0,0,0,0])
print(a_ij)

# X_j column 
def find_X_j(series):

    arr1 = list(series)
    arr2 = np.zeros(6)
    while True:
        L = arr1.index(max(arr1))
        arr2[L]=1
        if arr2.sum()==2:
            break
        arr1[L]=-inf
      
    return arr2

#LB column
# X_j가 배정됨에 따라 커버되는 new_Zi 를 구하는 것은 그냥 a_ij의 i 번째 행을 갖다 쓰면 된다..
def LB():
    arr=[]
    for n,i in enumerate(df['X_j']):
        if i==1:
            sigma = np.dot(a_ij[n],df['h_i'])
            arr.append(sigma)
        elif i==0:
            arr.append(0)
    return arr
    


# Dataframe 생성
column_list = ['h_i','lambda_i','Z_i','(h_i-lambda_i)*Z_i','sigma(a*lambda)','X_j','sigma(a*lambda)X_j','sigma(aX)','sigma(aX)-Z','LB']
index_list = ['A','B','C','D','E','F']
table = np.zeros((len(index_list),len(column_list)))

df = pd.DataFrame(table)
df.index = index_list
df.columns=column_list

#초기값 생성
df['h_i'] = demand
h_bar = df['h_i'].mean()
df['lambda_i']=np.array([h_bar+0.5*(i-h_bar) for i in df['h_i']])
df['Z_i']= np.where(df['h_i']-df['lambda_i']>=0,1,0)
df['(h_i-lambda_i)*Z_i'] = (df['h_i']-df['lambda_i'])*df['Z_i']
df['sigma(a*lambda)'] = np.dot(a_ij,df['lambda_i'])
df['X_j'] = find_X_j(df['sigma(a*lambda)'])
df['sigma(a*lambda)X_j'] = df['sigma(a*lambda)']*df['X_j']
df['sigma(aX)']= np.dot(a_ij,df['X_j'])
df['sigma(aX)-Z']=df['sigma(aX)']-df['Z_i']
df['LB']=LB()

upper_bound.append((df['(h_i-lambda_i)*Z_i']+df['sigma(a*lambda)X_j']).sum())
lower_bound.append(df['LB'].max())
print(upper_bound,lower_bound)

df

[[1 1 0 1 0 0]
 [1 1 0 1 0 0]
 [0 0 1 0 1 0]
 [1 1 0 1 0 0]
 [0 0 1 0 1 0]
 [0 0 0 0 0 1]]
[114.5] [36]


,h_i,lambda_i,Z_i,(h_i-lambda_i)*Z_i,sigma(a*lambda),X_j,sigma(a*lambda)X_j,sigma(aX),sigma(aX)-Z,LB
A,10,15.0,0,-0.0,48.0,1.0,48.0,2.0,2.0,36
B,8,14.0,0,-0.0,48.0,1.0,48.0,2.0,2.0,36
C,22,21.0,1,1.0,34.5,0.0,0.0,0.0,-1.0,0
D,18,19.0,0,-0.0,48.0,0.0,0.0,2.0,2.0,0
E,7,13.5,0,-0.0,34.5,0.0,0.0,0.0,0.0,0
F,55,37.5,1,17.5,37.5,0.0,0.0,0.0,-1.0,0


6